# Variational quantum eigensolver
A tool to address a specific class of combinatorial problems. VQEs share with quantum machine learning techniques the main driving idea of using classical optimization to minimize the expectation of an observable as the cost function.

It is a quantum algorithm that can be used to find the lowest-energy eigenstate (i.e., the ground state) of a quantum system. The VQE algorithm is a hybrid quantum-classical algorithm, meaning that it combines the use of a quantum computer with classical computation. The VQE algorithm works by iteratively finding the parameters of a trial wave function that best approximate the ground state of the quantum system. The trial wave function is a mathematical function that is used to represent the state of the quantum system, and it is parameterized by a set of parameters that can be adjusted to optimize the wave function.

The VQE algorithm uses a quantum computer to prepare the trial wave function and measure the energy of the wave function, and classical computation to optimize the parameters of the wave function based on the energy measurement. The optimization process is repeated until the ground state of the quantum system is found to a desired accuracy. It has the potential to enable the simulation of quantum systems that are otherwise too complex to be simulated using classical computers.

**Problem**: Calculate the ground state energy of a $H_{3}$ molecule to understand VQE <br>
Steps: <br>
1. Find molecular Hamiltonian
2. Prepare trial ground state using ansatz
3. Minimize the expectation value of Hamiltonian$(<\hat{H}>)$

In [47]:
import pennylane as qml
from pennylane import numpy as np
from pennylane import qchem

In [48]:
# Step-1
symbols = ["H", "H", "H"]
coordinates = np.array([[0.0102, 0.0442, 0.0], [0.9867, 1.6303, 0.0], [1.8720, -0.0085, 0.0]])

In [49]:
# instead code for H3 ion as its easier to work with 2 electrons than with 3
hamiltonian, qubits = qchem.molecular_hamiltonian(symbols, coordinates, charge=1)  # define molecular hamiltonian; also calculates the number of qubits that we need for our model

In [50]:
print(qubits)

6


In [51]:
#define an approximate ground state known as Hartree-Fock State
hf = qchem.hf_state(electrons=2, orbitals=6)
print(hf)

[1 1 0 0 0 0]


In [52]:
num_wires = qubits
dev = qml.device("default.qubit", wires=num_wires)

In [53]:
@qml.qnode(dev)
def exp_energy(state):
    qml.BasisState(np.array(state),wires=range(num_wires)) #BasisState operation to initialize the qubit register
    return qml.expval(hamiltoninan)

In [54]:
exp_energy(hf)

tensor(-1.24655016, requires_grad=True)

we get an energy of -1.24, we will see that this is really not the ground energy so it is not the ground state

$\newcommand{\ket}[1]{\left|{#1}\right\rangle}$
$\newcommand{\bra}[1]{\left\langle{#1}\right|}$
**Double Excitation gate**: takes one parameter $\theta$ and it takes 4 input qubits, say 1,1,0,0. Then what this gate will do is take it into a superposition: $\cos{\frac{\theta}{2}}\ket{1100}-\sin{\frac{\theta}{2}}\ket{0011}$ 

It takes the lower energy electrons into a superposition with themselves and the higher levels and also de-excites. So if the initial state was $\ket{0011}$, it would take it to $\cos{\frac{\theta}{2}}\ket{0011}+\sin{\frac{\theta}{2}}\ket{1100}$ 

ansatz for the ground state: we are working with 6qubits so it should be a superposition between the Hartree-Fock State(initial state), the state with two electrons in the second energy level and the state with two electrons in the highest energy level <br>
ansatz: $\alpha \ket{110000}+\beta \ket{001100}+\gamma \ket{000011}$ <br>
Why this ansatz? In the Hartree-Fock approximation, we neglect some interactions between electrons which are strongest when the electrons are in the same molecular energy level, for this reason this ansatz accounts for the error made in the Hartree-Fock approximation <br>
trial state: $\ket{\psi}= \cos{\frac{\theta_{1}}{2}}(\cos{\frac{\theta_{2}}{2}}\ket{110000}-\sin{\frac{\theta_{2}}{2}}\ket{000011})-\sin{\frac{\theta_{1}}{2}}\ket{001100}$

In [55]:
# Step-2
# double excitation gate prepares the candidate ground state
def ansatz(params): #params: parameters of the double excitation gate
    qml.BasisState(hf, wires=range(num_wires)) # initial state
    qml.DoubleExcitation(params[0], wires=[0,1,2,3])  # first double excitation gate on qubits 0,1,2,3 as it is taking the electrons in the lowest energy level to the intermediate energy level
    qml.DoubleExcitation(params[1], wires=[0,1,4,5])  # second double excitation gate on qubits 0,1,4,5 as it will take the electrons in the lowest energy level to the highest energy level 

To find the ground state, we need to make use of a property known as the **Ritz Variational principle** which tells that the state that minimizes the expectation value of the Hamiltonian is the ground state; $E_{0}\leq min\bra{\psi}\hat{H}\ket{\psi}$: cost function

In [56]:
# Step-3
@qml.qnode(dev)
def cost_function(params):
    ansatz(params) # prepare the candidate ground state
    return qml.expval(hamiltonian)

In [57]:
cost_function([0.1, 0.1])

tensor(-1.26796721, requires_grad=True)

we get an energy of -1.26 < -1.24 for the Hartree-Fock state, already found a state that has a lower energy of what was supposed to be the ground state

In [58]:
# minimize the expectation value of the Hamiltonian: Optimize the Cost function ckt
opt = qml.GradientDescentOptimizer(stepsize=0.4)
theta = np.array([0.0,0.0], requires_grad=True)

energy = [cost_function(theta)]
angle = [theta]
max_iterations = 20

for n in range(max_iterations):
    theta, prev_energy = opt.step_and_cost(cost_function, theta)
    
    energy.append(cost_function(theta))
    angle.append(theta)
    
    if n%2==0:
        print(f"Step={n}, Energy={energy[-1]:.8f} Ha")

Step=0, Energy=-1.26070025 Ha
Step=2, Energy=-1.27115671 Ha
Step=4, Energy=-1.27365804 Ha
Step=6, Energy=-1.27425241 Ha
Step=8, Energy=-1.27439362 Ha
Step=10, Energy=-1.27442718 Ha
Step=12, Energy=-1.27443517 Ha
Step=14, Energy=-1.27443707 Ha
Step=16, Energy=-1.27443752 Ha
Step=18, Energy=-1.27443763 Ha


In [59]:
print("\n" f"Final ground energy: {energy[-1]:.8f} Ha")
print("\n" f"Final angle parameters: {theta[0]:.8f} {theta[1]:.8f}")


Final ground energy: -1.27443764 Ha

Final angle parameters: 0.19203468 0.19290335


In [60]:
@qml.qnode(dev)
def ground_state(params):
    ansatz(params)
    return qml.state()

In [61]:
ground_state(theta)

tensor([ 0.        +0.j,  0.        +0.j,  0.        +0.j,
        -0.09585862+0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
        -0.09586987+0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.99076743+0.j,  0.        +0.j,  0.        +0.

the above state corresponds to: <br>
$\ket{ground-state} = 0.9908\ket{110000}-0.096\ket{001100}-0.096\ket{000011}$